In [1]:
PATH = "../../data/labels/"
GT   = "../../data/ground_truth.json"
OUT  = "../../data/cpachecker_labels.jsonl"


In [2]:
import os
import json
from glob import glob

with open(GT, "r") as i:
    gt = json.load(i)

def is_correct(object):
    file = object["file"]
    file = os.path.basename(file)
    file, _ = os.path.splitext(file)
    fgt  = gt.get(file, None)
    if fgt is None: print(file);return False
    return fgt

def _path_to_task_id(path):
    if len(path.split("/")) == 2: return path
    base_name = os.path.basename(path)
    dirname = os.path.basename(os.path.dirname(path))
    base_name = base_name.replace(".c", "").replace(".i", "").replace(".yml", "")
    return f"{dirname}/{base_name}"

output = {}

# {"tool": "CBMC", "benchmark": "cbmc", "task_file": "../sv-benchmarks/c/systemc/bist_cell.cil.yml", "status": "ERROR (42)", "verdict": "true", "cputime": 875.252759525, "walltime": 875.064540537016, "memory": 11448655872}

for result_file in glob(os.path.join(PATH, "*.json")):
    name = os.path.basename(result_file).replace(".json", "")
    with open(result_file, "r") as i:
        content = json.load(i)
    
    for object in content.values():
        identifier = object["file"]
        identifier = os.path.basename(identifier).replace(".c", "").replace(".i", "").replace(".yml", "")
        if identifier not in output:
            output[identifier] = []

        file_id = _path_to_task_id(object["file"])

        result = {
            "tool": name,
            "benchmark": name,
            "task_file": file_id,
            "status"   : object["status"],
            "verdict"  : is_correct(object),
            "cputime"  : object["cputime"],
            "memory"   : object["memUsage"]
        }

        output[identifier].append(result)

In [3]:
with open(OUT, "w") as o:
    for results in sorted(output.values(), key = lambda x: len(x)):
        if len(results) == 1: continue
        o.write(json.dumps(results) + "\n")